In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import requests
import pymongo
import re

# Google API Key
from config import gkey

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"


# San Juan 18.466333,-66.105721
# Ponce 18.01108, -66.61406
# Rincon 18.340216, -67.250015
# Fajardo 18.325787, -65.652382
# Arecibo 18.47245, -66.71573

# spanish, puertorican, middle eastern, chinese, fast food, bar restaurant
params = {
    "location": "18.466333,-66.105721",  # San Juan coords
    "rankby": "distance",
    "type": "hotel",
    "key": gkey,
    "keyword": ""
}

response = requests.get(base_url, params=params).json()
response

results = response['results']
results

[{'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 18.466333, 'lng': -66.105721},
   'viewport': {'northeast': {'lat': 18.4676399802915,
     'lng': -66.10436841970849},
    'southwest': {'lat': 18.4649420197085, 'lng': -66.1070663802915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/generic_business-71.png',
  'name': 'RightClick Media Inc.',
  'place_id': 'ChIJV43hytxuA4wROHsBjdRPrO8',
  'reference': 'ChIJV43hytxuA4wROHsBjdRPrO8',
  'scope': 'GOOGLE',
  'types': ['electronics_store',
   'home_goods_store',
   'point_of_interest',
   'store',
   'establishment'],
  'vicinity': '100 Carmen Hills Drv, Apt 132, San Juan'},
 {'business_status': 'OPERATIONAL',
  'geometry': {'location': {'lat': 18.466333, 'lng': -66.10571999999999},
   'viewport': {'northeast': {'lat': 18.4676399802915,
     'lng': -66.1043674197085},
    'southwest': {'lat': 18.4649420197085, 'lng': -66.1070653802915}}},
  'icon': 'https://maps.gstatic.com/mapfiles/place_api

In [5]:
for row in results: 
    text = row["photos"][0]["html_attributions"][0]
    url = re.findall(r'"(.*?)"', text)
    
    info = f"""
    Name:  {row["name"]}
    URL:  {url[0]}
    Rating:  {row["rating"]}
    Address:  {row["vicinity"]}
    """
# create code that will insert into a sql data.    
#     dic = {
#         'Name': row["name"],
#         'Type': "Chinese",
#         'Address': row['vicinity']
#     }
    
    print(info)    


    Name:  Han Cream
    URL:  https://maps.google.com/maps/contrib/100796978647196078569
    Rating:  3.7
    Address:  201 Ca. Tanca, San Juan
    

    Name:  Cathay
    URL:  https://maps.google.com/maps/contrib/109078591550070106842
    Rating:  4
    Address:  410 Avenida de la Constitución, San Juan
    

    Name:  Osaka Japanese and Chinese Restaurant
    URL:  https://maps.google.com/maps/contrib/112888380375138601484
    Rating:  4.4
    Address:  670 Avenida de la Constitución, San Juan
    

    Name:  New Taste
    URL:  https://maps.google.com/maps/contrib/114517340627932088149
    Rating:  4.3
    Address:  1051 Ashford Ave, San Juan
    

    Name:  Jo-King
    URL:  https://maps.google.com/maps/contrib/105172100292311379702
    Rating:  4.5
    Address:  PR-24, Guaynabo
    

    Name:  China Sun
    URL:  https://maps.google.com/maps/contrib/102324180717088381318
    Rating:  4.3
    Address:  00907, 1213 Avenida de la Constitución, San Juan
    

    Name:  La Chin

KeyError: 'photos'